# tweet grab

In [3]:
import os
import tweepy
import logging
import pprint
import json
import time
import boto3
import datetime as dt
import pandas as pd
import numpy as np
import botocore.session
from botocore.exceptions import ClientError
from boto3.dynamodb.conditions import Key, Attr
print('imported modules successfully.')

imported modules successfully.


### set api creds

In [4]:
base = os.getcwd()
data_dir = os.path.join(base, 'data')
log_dir = os.path.join(base, 'logs')
pp = pprint.PrettyPrinter(indent = 1)

os.chdir(base)

with open('./../creds.json', 'r') as f:
    tweepy_creds = json.load(f)
f.close()
    
auth = tweepy.OAuthHandler(tweepy_creds['twitter-api-key'], tweepy_creds['twitter-secret-key'])
auth.set_access_token(tweepy_creds['twitter-access-token'], tweepy_creds['twitter-secret-access'])

api = tweepy.API(auth)
logging.debug('authorized API w/ client id and secret.')

client = boto3.client('dynamodb',
  endpoint_url = "http://{}:{}".format(tweepy_creds["host"], tweepy_creds["port"]),
  aws_access_key_id=tweepy_creds["aws-access-key"],
  aws_secret_access_key=tweepy_creds["aws-secret-access-key"],
  region_name='us-east-2')

dynamodb = boto3.resource('dynamodb', \
            endpoint_url = "http://{}:{}".format(tweepy_creds["host"], tweepy_creds["port"]), \
            region_name='us-east-2')

session = botocore.session.get_session()
dynamodb_session = session.create_client('dynamodb', \
                    region_name='us-east-2',
                    endpoint_url = "http://{}:{}".format(tweepy_creds["host"], tweepy_creds["port"]),
                    aws_access_key_id=tweepy_creds["aws-access-key"],
                    aws_secret_access_key=tweepy_creds["aws-secret-access-key"]) # low-level client

tweets = dynamodb.Table("tweets")

print("created dynamo client.")

created dynamo client.


### get followers

In [5]:
def get_followers(user):
    following = []
    friends_response = api.get_friends(screen_name = user, count = 100, cursor = -1)
    following.extend([t.screen_name for t in friends_response[0]])
    next_cursor = friends_response[1][1]
    print("got most recent 100 following")

    while next_cursor != 0:
        try:
            friends_response = api.get_friends(screen_name = user, count = 100, cursor = next_cursor)
            following.extend([t.screen_name for t in friends_response[0]])
            next_cursor = friends_response[1][1]
            print("got next 100 following")
        
        except:
            time.sleep(60*15)

    return(following)

In [6]:
my_following = get_followers("xanax_princess_")

got most recent 100 following
got next 100 following
got next 100 following
got next 100 following


In [7]:
print(len(my_following))

387


In [17]:
# test = api.user_timeline(screen_name = my_following[0], count = 25)

In [8]:
# m = len(test)
# test[m-1]._json["id"]

In [9]:
def batch_put_response(response, table):
    with table.batch_writer() as batch:
        for r in response:
            try:
                batch.put_item(Item = {
                    "user": r.user.screen_name,
                    "created_at": r.created_at.strftime("%Y-%m-%dT%H:%M:%SZ"),
                    "tweet": r.text,
                    "id": r.id,
                    "retweets": r.retweet_count,
                    "favorites": r.favorite_count,
                    "inserted_at": dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
                })
        
            except ClientError as err:
                print("Couldn't load data into table %s. Here's why: %s: %s", table.name, err.response['Error']['Code'], err.response['Error']['Message'])
                raise

### get tweets per user

In [10]:
def get_tweets(user, table):
    print("scraping tweets for user: {}".format(user))
    timeline_response = api.user_timeline(screen_name = user, count = 25)
    n = len(timeline_response)
    last_id = timeline_response[n-1]._json["id"]
    i = 0

    print("putting first batch")
    batch_put_response(timeline_response, table)
    print("inserted batch to dynamodb successfully.")
    i += 1

    while i < 25:
        try:
            timeline_response = api.user_timeline(screen_name = user, count = 25, max_id = last_id)
            n = len(timeline_response)
            last_id = timeline_response[n-1]._json["id"]

            batch_put_response(timeline_response, table)
            print("inserted batch to dynamodb successfully.")
            i += 1
    
        except Exception as e:
            print(e)
            if "429" in str(e):
                print("sleeping for 5 minutes")
                time.sleep(60*5)

            else:
                print("sleeping for a minute")
                time.sleep(60)

In [11]:
# timeline_response = api.user_timeline(screen_name = my_following[0], count = 100)
# timeline_response[0]._json

## get all tweets

In [2]:
tweets = dynamodb.Table("tweets")

for f in my_following:
    try:
        output = get_tweets(f, tweets)
        time.sleep(5)

    except KeyboardInterrupt as k:
        print(k)

NameError: name 'dynamodb' is not defined

In [55]:
response = tweets.scan()
response

{'Items': [{'favorites': Decimal('3'),
   'created_at': '2023-01-24T18:09:56Z',
   'tweet': '16 years \U0001fae0 https://t.co/pmXRoXOdWz',
   'id': Decimal('1617947826671190017'),
   'retweets': Decimal('0'),
   'user': 'colborkmundus',
   'inserted_at': '2023-01-24T21:54:20Z'},
  {'favorites': Decimal('1'),
   'created_at': '2023-01-24T18:23:49Z',
   'tweet': '16 years, noooooooo https://t.co/8poAs8c6dr',
   'id': Decimal('1617951319050039299'),
   'retweets': Decimal('0'),
   'user': 'colborkmundus',
   'inserted_at': '2023-01-24T21:54:20Z'},
  {'favorites': Decimal('4'),
   'created_at': '2023-01-24T19:51:02Z',
   'tweet': 'gosh this stupid hellsite (affectionate) is my favorite bar and all and i love doing shitpost jello shots with the… https://t.co/2PP4R0jnJC',
   'id': Decimal('1617973267859673088'),
   'retweets': Decimal('0'),
   'user': 'colborkmundus',
   'inserted_at': '2023-01-24T21:54:20Z'},
  {'favorites': Decimal('0'),
   'created_at': '2023-01-24T23:56:36Z',
   'tweet':

In [56]:
! aws dynamodb describe-table --table-name "tweets" --endpoint-url http://xanaxprincess.asuscomm.com:8001

{
    "Table": {
        "AttributeDefinitions": [
            {
                "AttributeName": "user",
                "AttributeType": "S"
            },
            {
                "AttributeName": "created_at",
                "AttributeType": "S"
            }
        ],
        "TableName": "tweets",
        "KeySchema": [
            {
                "AttributeName": "user",
                "KeyType": "HASH"
            },
            {
                "AttributeName": "created_at",
                "KeyType": "RANGE"
            }
        ],
        "TableStatus": "ACTIVE",
        "CreationDateTime": 1674535588.165,
        "ProvisionedThroughput": {
            "LastIncreaseDateTime": 0.0,
            "LastDecreaseDateTime": 0.0,
            "NumberOfDecreasesToday": 0,
            "ReadCapacityUnits": 10,
            "WriteCapacityUnits": 10
        },
        "TableSizeBytes": 5434,
        "ItemCount": 25,
        "TableArn": "arn:aws:dynamodb:ddblocal:000000000000:tab